# Description

This notebook contains code taken from the MultiPLIER repo to download and process recount2 data.
The code was taken from [here](https://github.com/greenelab/rheum-plier-data/blob/master/recount2/1-get_all_recount_dataset.R) and [here](https://github.com/greenelab/rheum-plier-data/blob/master/recount2/2-prep_recount_for_plier.R).

The output are Python pickle files with a large matrix with genes in rows and samples in columns, and another file with gene ID mappings.

# Modules

In [ ]:
`%>%` <- dplyr::`%>%`
library(biomaRt)
library(reticulate)

# Settings

In [ ]:
recount2full.data.dir <- Sys.getenv("CM_RECOUNT2FULL_DATA_DIR")

In [ ]:
recount2full.data.dir

In [ ]:
dir.create(recount2full.data.dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
data.dir <- Sys.getenv("CM_RECOUNT2FULL_INTERNAL_DATA_DIR")

In [ ]:
data.dir

In [ ]:
dir.create(data.dir, recursive = TRUE, showWarnings = FALSE)

# Load raw data

In [ ]:
input.file <- file.path(recount2full.data.dir, "recount2_rpkm_raw.rds")

In [ ]:
input.file

In [ ]:
rpkm.df <- readRDS(input.file)

In [ ]:
dim(rpkm.df)

In [ ]:
head(rpkm.df[, 1:10])

# Preprocess data

In [ ]:
# Transform ensembl id to genesymbol
mart <- biomaRt::useDataset(
  "hsapiens_gene_ensembl",
  biomaRt::useMart("ensembl")
)

In [ ]:
genes <- unlist(lapply(strsplit(rpkm.df$ENSG, "[.]"), `[[`, 1))

rpkm.df$ensembl_gene_id <- unlist(lapply(
  strsplit(rpkm.df$ENSG, "[.]"),
  `[[`, 1
))

gene.df <- biomaRt::getBM(
  filters = "ensembl_gene_id",
  attributes = c("ensembl_gene_id", "hgnc_symbol"),
  values = genes,
  mart = mart
)

In [ ]:
# filter to remove genes without a gene symbol
gene.df <- gene.df %>% dplyr::filter(complete.cases(.))

In [ ]:
# add gene symbols to expression df
rpkm.df <- dplyr::inner_join(gene.df, rpkm.df,
  by = "ensembl_gene_id"
)

In [ ]:
# keep gene mappings
gene.df <- rpkm.df %>% dplyr::select(ensembl_gene_id, hgnc_symbol)

In [ ]:
dim(gene.df)

In [ ]:
head(gene.df)

In [ ]:
# set Ensemble IDs as rownames
rownames(rpkm.df) <- make.names(rpkm.df$ensembl_gene_id, unique = TRUE)

In [ ]:
# remove gene identifier columns
rpkm.df <- rpkm.df %>% dplyr::select(-c(ensembl_gene_id:ENSG))

In [ ]:
dim(rpkm.df)

In [ ]:
head(rpkm.df[, 1:10])

# Save

## Gene ID mappings

In [ ]:
output_filepath <- file.path(recount2full.data.dir, "recount2_gene_ids_mappings")

In [ ]:
output_filepath

In [ ]:
saveRDS(gene.df, file = paste0(output_filepath, ".rds"))

In [ ]:
py_save_object(gene.df, paste0(output_filepath, ".pkl"))

## Gene expression data

In [ ]:
output_filepath <- file.path(recount2full.data.dir, "recount2_rpkm")

In [ ]:
output_filepath

In [ ]:
saveRDS(rpkm.df, file = paste0(output_filepath, ".rds"))

In [ ]:
py_save_object(rpkm.df, paste0(output_filepath, ".pkl"))

# Cleanup

In [ ]:
# the raw file is not longer necessary
if (file.exists(input.file)) {
  # Delete file if it exists
  file.remove(input.file)
}